# Step 1 Clean the synthetic dataset

*   The sythetic dataset uses llama embedding and k-means clustering, with 9:1 Ratio: 2 of 60%, 2 of 70%, 2 of 80%, 2 of 90% randomly selected symptoms and 1 of 100% case.
*   Remove the cases with 0 symptom in the generated synthetic dataset.
*   Divide all cases into 80% and 20%



In [1]:
import os, json, math, random, itertools
from collections import defaultdict, Counter
from pathlib import Path
import pandas as pd

from google.colab import drive
drive.mount('/content/drive')

!ls "/content/drive/MyDrive/Stanford/CS229/Final Project/llama+kmeans_synthetic_cases_20251029-044735"

Mounted at /content/drive
 cluster_centroids_mxbai_10000.json	    synthetic_data.json
 label_map.json				    synthetic_data_summary_table.csv
 PUBMED_cohort.xlsx			    synthetic_data_summary_table.gsheet
 pubmed_symptom_cluster_assignments.csv     train.jsonl
 pubmed_symptom_embeddings_dict_mxbai.pkl  'Untitled document.gdoc'
 split_summary.csv			    val.jsonl
 split_summary.gsheet			    xgb_checkpoints
 symptom_vocab.json			    XGBoost_Ximing_08Nov25.ipynb


In [ ]:
from pathlib import Path
import random
# -------- CONFIG --------
INPUT_PATH = Path("/content/drive/MyDrive/Stanford/CS229/Final Project/llama+kmeans_synthetic_cases_20251029-044735/synthetic_data.json")
TRAIN_OUT  = Path("/content/drive/MyDrive/Stanford/CS229/Final Project/llama+kmeans_synthetic_cases_20251029-044735/train.jsonl")
VAL_OUT    = Path("/content/drive/MyDrive/Stanford/CS229/Final Project/llama+kmeans_synthetic_cases_20251029-044735/val.jsonl")
SUMMARY_CSV= Path("/content/drive/MyDrive/Stanford/CS229/Final Project/llama+kmeans_synthetic_cases_20251029-044735/split_summary.csv")

GLOBAL_SEED = 42
random.seed(GLOBAL_SEED)
# ------------------------
assert INPUT_PATH.exists(), f"File not found: {INPUT_PATH.resolve()}"
!head -n 5 "/content/drive/MyDrive/Stanford/CS229/Final Project/llama+kmeans_synthetic_cases_20251029-044735/synthetic_data.json"


{"syndrome": "DEVELOPMENTAL AND EPILEPTIC ENCEPHALOPATHY 9; DEE9", "coverage_pct": 60, "case_type": "60%", "rep": 1, "n_total_symptoms": 17, "n_selected": 11, "symptom_ids": [77, 109, 249, 717, 870, 2537, 6069, 6267, 6959, 7881, 9751]}
{"syndrome": "DEVELOPMENTAL AND EPILEPTIC ENCEPHALOPATHY 9; DEE9", "coverage_pct": 60, "case_type": "60%", "rep": 2, "n_total_symptoms": 17, "n_selected": 11, "symptom_ids": [77, 109, 717, 962, 2537, 6069, 6267, 6959, 7881, 8951, 9009]}
{"syndrome": "DEVELOPMENTAL AND EPILEPTIC ENCEPHALOPATHY 9; DEE9", "coverage_pct": 70, "case_type": "70%", "rep": 1, "n_total_symptoms": 17, "n_selected": 12, "symptom_ids": [77, 109, 249, 870, 962, 1027, 1990, 6069, 6959, 7881, 9009, 9751]}
{"syndrome": "DEVELOPMENTAL AND EPILEPTIC ENCEPHALOPATHY 9; DEE9", "coverage_pct": 70, "case_type": "70%", "rep": 2, "n_total_symptoms": 17, "n_selected": 12, "symptom_ids": [77, 109, 717, 870, 962, 1027, 6069, 6090, 6267, 8951, 9009, 9751]}
{"syndrome": "DEVELOPMENTAL AND EPILEPTIC E

In [ ]:
import os, json, math, random, itertools
from collections import defaultdict, Counter
import pandas as pd

# === Load (JSON first; if it fails, parse as JSONL) ===
def load_json_or_jsonl(path: Path):
    try:
        with open(path, "r", encoding="utf-8") as f:
            return json.load(f)  # a single JSON value (list or dict)
    except json.JSONDecodeError:
        # Fallback: JSON Lines
        rows = []
        with open(path, "r", encoding="utf-8") as f:
            for ln in f:
                ln = ln.strip()
                if not ln:
                    continue
                try:
                    rows.append(json.loads(ln))
                except json.JSONDecodeError:
                    # skip non-JSON lines if any
                    pass
        return rows

data = load_json_or_jsonl(INPUT_PATH)


In [ ]:
import json, math, random
from collections import defaultdict, Counter
import pandas as pd

GLOBAL_SEED = 42
random.seed(GLOBAL_SEED)

# --- Load as JSONL (since your generator writes one record per line) ---
with open(INPUT_PATH, "r", encoding="utf-8") as f:
    data = [json.loads(ln) for ln in f if ln.strip()]

# --- Normalize into {syndrome, symptoms} keys your downstream expects ---
def get_symptoms(d):
    for k in ("symptoms","symptom_ids","features","codes"):
        if k in d:
            return d[k]
    return None

records = []
for r in data:
    syn = r.get("syndrome") or r.get("label") or r.get("disease") or r.get("name") or r.get("condition")
    syms = get_symptoms(r)
    if syn is not None and syms:  # <-- empty list will return False here
        records.append({"syndrome": str(syn), "symptoms": list(syms)})

print(f"Loaded {len(records)} usable records after first filtering.")

# --- Remove zero-symptom cases ---
records = [r for r in records if isinstance(r["symptoms"], list) and len(r["symptoms"]) > 0]
print(f"Loaded {len(records)} usable records after second filtering.")


Loaded 14940 usable records after first filtering.
Loaded 14940 usable records after second filtering.


# Step 2 Generate the 80/20 training/validation dataset

*   Each syndrome has 9 samples, split into 80/20 for training/validation. (7 cases for training and 2 cases for validation)



In [ ]:
# --- Group by syndrome ---
group_by_syndrome = defaultdict(list)
for r in records:
    group_by_syndrome[r["syndrome"]].append(r)

# --- Per-syndrome 80/20 split (floor to train; keep both non-empty if n>=2) ---
train, val = [], []
for syn, cases in group_by_syndrome.items():
    rnd = random.Random(GLOBAL_SEED + (hash(syn) % 1_000_000))
    shuffled = cases[:]
    rnd.shuffle(shuffled)
    n = len(shuffled)
    n_train = max(1, int(math.floor(0.8 * n))) if n > 1 else 1
    if n >= 2 and n_train == n:
        n_train = n - 1
    train.extend(shuffled[:n_train])
    val.extend(shuffled[n_train:])

def write_jsonl(path, rows):
    with open(path, "w", encoding="utf-8") as f:
        for r in rows:
            f.write(json.dumps(r, ensure_ascii=False) + "\n")

write_jsonl(TRAIN_OUT, train)
write_jsonl(VAL_OUT, val)

# --- Summary CSV ---
def summarize(rows):
    c = Counter(r["syndrome"] for r in rows)
    return pd.DataFrame([{"syndrome": k, "n": v} for k, v in sorted(c.items(), key=lambda x: (-x[1], x[0]))])

summary_all = summarize(records).rename(columns={"n":"n_total"})
summary_tr  = summarize(train).rename(columns={"n":"n_train"})
summary_va  = summarize(val).rename(columns={"n":"n_val"})
summary = summary_all.merge(summary_tr, on="syndrome", how="left").merge(summary_va, on="syndrome", how="left")
summary[["n_train","n_val"]] = summary[["n_train","n_val"]].fillna(0).astype(int)
summary["train_ratio"] = (summary["n_train"] / summary["n_total"]).round(3)
summary["val_ratio"]   = (summary["n_val"]   / summary["n_total"]).round(3)
summary = summary.sort_values(["n_total","syndrome"], ascending=[False, True]).reset_index(drop=True)
summary.to_csv(SUMMARY_CSV, index=False)

print("✅ Preprocessing done.")
print("Train:", len(train), "→", TRAIN_OUT)
print("Val:  ", len(val),   "→", VAL_OUT)
print("Summary CSV:", SUMMARY_CSV)

summary.head(10)

✅ Preprocessing done.
Train: 11620 → /content/drive/MyDrive/Stanford/CS229/Final Project/llama+kmeans_synthetic_cases_20251029-044735/train.jsonl
Val:   3320 → /content/drive/MyDrive/Stanford/CS229/Final Project/llama+kmeans_synthetic_cases_20251029-044735/val.jsonl
Summary CSV: /content/drive/MyDrive/Stanford/CS229/Final Project/llama+kmeans_synthetic_cases_20251029-044735/split_summary.csv


,syndrome,n_total,n_train,n_val,train_ratio,val_ratio
0,"2,4-DIENOYL-CoA REDUCTASE DEFICIENCY; DECRD",9,7,2,0.778,0.222
1,2-METHYLBUTYRYL-CoA DEHYDROGENASE DEFICIENCY,9,7,2,0.778,0.222
2,3-HYDROXY-3-METHYLGLUTARYL-CoA LYASE DEFICIENC...,9,7,2,0.778,0.222
3,3-HYDROXYACYL-CoA DEHYDROGENASE DEFICIENCY,9,7,2,0.778,0.222
4,3-HYDROXYISOBUTYRYL-CoA HYDROLASE DEFICIENCY; ...,9,7,2,0.778,0.222
5,3-METHYLCROTONYL-CoA CARBOXYLASE 1 DEFICIENCY;...,9,7,2,0.778,0.222
6,3-METHYLCROTONYL-CoA CARBOXYLASE 2 DEFICIENCY;...,9,7,2,0.778,0.222
7,"3-METHYLGLUTACONIC ACIDURIA WITH DEAFNESS, ENC...",9,7,2,0.778,0.222
8,"3-METHYLGLUTACONIC ACIDURIA, TYPE I; MGCA1",9,7,2,0.778,0.222
9,"3-METHYLGLUTACONIC ACIDURIA, TYPE IX; MGCA9",9,7,2,0.778,0.222


# Step 3: Vectorize - build bag-of-symptoms
*   Look at training data only → collect all unique symptom IDs = sym_vocab.
*   Convert each training example into a vector of 0s and 1s of length len(sym_vocab).
*   When converting validation examples, use the same sym_vocab. If a symptom appears in validation but not in training, we simply ignore it (treat as unseen).


---
Gives us:
*   A feature matrix X_train of shape [num_training_cases, num_unique_symptoms]
*   A label vector y_train with the syndrome for each row
*   **Symptom_vocab.json** saves the complete list of unique symptom IDs seen in the training dataset, in sorted order.
*   **label_map.json** maps each syndrome label (string name) to its numeric index used by the model.





In [ ]:
INPUT_PATH = Path("/content/drive/MyDrive/Stanford/CS229/Final Project/llama+kmeans_synthetic_cases_20251029-044735/synthetic_data.json")
TRAIN_OUT  = Path("/content/drive/MyDrive/Stanford/CS229/Final Project/llama+kmeans_synthetic_cases_20251029-044735/train.jsonl")
VAL_OUT    = Path("/content/drive/MyDrive/Stanford/CS229/Final Project/llama+kmeans_synthetic_cases_20251029-044735/val.jsonl")

In [ ]:
import json
from scipy.sparse import csr_matrix
import numpy as np
from sklearn.preprocessing import LabelEncoder
from pathlib import Path

def read_jsonl(path):
    rows = []
    with open(path, "r", encoding="utf-8") as f:
        for ln in f:
            if ln.strip():
                rows.append(json.loads(ln))
    return rows

train_rows = read_jsonl(TRAIN_OUT)
val_rows   = read_jsonl(VAL_OUT)

# Build symptom vocabulary from TRAINING dataset ONLY (avoid leakage)
sym_vocab = sorted({sid for r in train_rows for sid in r["symptoms"]})
sym_index = {sid:i for i, sid in enumerate(sym_vocab)}

def rows_to_bow(rows):
    indptr, indices, data = [0], [], []
    labels = []
    for r in rows:
        cols = [sym_index[sid] for sid in r["symptoms"] if sid in sym_index]
        cols = sorted(set(cols))
        indices.extend(cols)
        data.extend([1]*len(cols))
        indptr.append(len(indices))
        labels.append(r["syndrome"])
    X = csr_matrix((data, indices, indptr), shape=(len(rows), len(sym_vocab)), dtype=np.float32)
    return X, labels

X_train, y_train = rows_to_bow(train_rows)
X_val,   y_val   = rows_to_bow(val_rows)

le = LabelEncoder().fit(y_train)
y_train_enc = le.transform(y_train)
y_val_enc   = le.transform(y_val)  # safe since per-syndrome split keeps labels present in train

print("X_train:", X_train.shape, "X_val:", X_val.shape, "classes:", len(le.classes_))

# Save vocab & label map
BASE_DIR = INPUT_PATH.parent

VOCAB_PATH = BASE_DIR / "symptom_vocab.json"
LABEL_MAP_PATH = BASE_DIR / "label_map.json"

with open(VOCAB_PATH, "w", encoding="utf-8") as f:
    json.dump(sym_vocab, f)

with open(LABEL_MAP_PATH, "w", encoding="utf-8") as f:
    json.dump({cls: int(i) for i, cls in enumerate(le.classes_)}, f, ensure_ascii=False)

print("Saved:", VOCAB_PATH)
print("Saved:", LABEL_MAP_PATH)

X_train: (11620, 9883) X_val: (3320, 9883) classes: 1660
Saved: /content/drive/MyDrive/Stanford/CS229/Final Project/llama+kmeans_synthetic_cases_20251029-044735/symptom_vocab.json
Saved: /content/drive/MyDrive/Stanford/CS229/Final Project/llama+kmeans_synthetic_cases_20251029-044735/label_map.json


# Step 4: Train XGBoost multi-class & evaluate
*   Use 5-fold cross-validation here.  
*   Because the dataset is highly imbalanced and tiny per class, we cannot just randomly split samples. Use a StratifiedKFold or StratifiedGroupKFold here to ensure every fold contains roughly the same class proportions.This prevents some folds from missing rare syndromes entirely.






In [ ]:
!python3 -m pip uninstall -y xgboost
!python3 -m pip install --no-cache-dir xgboost==3.1.1



Found existing installation: xgboost 2.1.1
Uninstalling xgboost-2.1.1:
  Successfully uninstalled xgboost-2.1.1
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.9/115.9 MB 154.7 MB/s eta 0:00:00


In [ ]:
import xgboost as xgb
print("XGBoost version:", xgb.__version__)  # should be 3.1.1 (Colab default right now)


XGBoost version: 3.1.1


In [ ]:
import os, re, glob, json, pickle
import numpy as np
from pathlib import Path
from collections import Counter
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score
import xgboost as xgb
from xgboost import callback

# ========= CONFIG =========
SAVE_DIR = Path("/content/drive/MyDrive/Stanford/CS229/Final Project/llama+kmeans_synthetic_cases_20251029-044735/xgb_checkpoints")
SAVE_DIR.mkdir(parents=True, exist_ok=True)

NUM_CLASS = len(le.classes_)     # from your LabelEncoder on y_train
N_BOOST_ROUNDS = 1000            # cap on total trees
EARLY_STOPPING_ROUNDS = 100      # stop if no improvement
CHECKPOINT_EVERY = 250           # save every N rounds
RANDOM_STATE = 42
N_SPLITS = 5
# ==========================

# ---- Version-agnostic periodic saver ----
class PeriodicSaver(callback.TrainingCallback):
    """Save booster every `every` rounds to SAVE_DIR / f'{name}_{iter:04d}.json'."""
    def __init__(self, save_dir: Path, name: str, every: int):
        self.save_dir = Path(save_dir)
        self.name = name
        self.every = int(every)

    def after_iteration(self, model, epoch: int, evals_log) -> bool:
        it = epoch + 1
        if it % self.every == 0:
            path = self.save_dir / f"{self.name}_{it:04d}.json"
            model.save_model(str(path))
            print(f"[checkpoint] saved {path}")
        return False  # continue training

def latest_checkpoint_for_fold(save_dir: Path, fold: int):
    """Return path to latest checkpoint (json) for this fold, or None."""
    pattern = str(save_dir / f"xgb_fold{fold}_*.json")
    candidates = glob.glob(pattern)
    if not candidates:
        return None
    best, best_iter = None, -1
    rx = re.compile(rf"xgb_fold{fold}_(\d+)\.json$")
    for p in candidates:
        m = rx.search(os.path.basename(p))
        if m:
            it = int(m.group(1))
            if it > best_iter:
                best_iter, best = it, p
    return best

# Adjust folds if the rarest class is too small
min_per_class = min(Counter(y_train_enc).values())
actual_splits = min(N_SPLITS, max(2, min_per_class))
print(f"Using {actual_splits}-fold CV (min per class = {min_per_class})")

kf = StratifiedKFold(n_splits=actual_splits, shuffle=True, random_state=RANDOM_STATE)
fold_accs = []

# Common XGBoost params
params = {
    "objective": "multi:softprob",
    "num_class": NUM_CLASS,
    "eval_metric": "mlogloss",
    "max_depth": 4,
    "learning_rate": 0.1,
    "subsample": 0.9,
    "colsample_bytree": 0.9,
    "reg_lambda": 1.0,
    "tree_method": "hist",
    "device": "cuda",
    "seed": RANDOM_STATE,
}

for fold, (train_idx, val_idx) in enumerate(kf.split(X_train, y_train_enc), 1):
    print(f"\n===== Fold {fold} =====")
    X_tr, X_va = X_train[train_idx], X_train[val_idx]
    y_tr, y_va = y_train_enc[train_idx], y_train_enc[val_idx]

    dtrain = xgb.DMatrix(X_tr, label=y_tr)
    dvalid = xgb.DMatrix(X_va, label=y_va)
    evals = [(dtrain, "train"), (dvalid, "valid")]

    resume_path = latest_checkpoint_for_fold(SAVE_DIR, fold)
    if resume_path:
        print(f"➡️  Resuming from checkpoint: {resume_path}")
    else:
        print("➡️  No checkpoint found; starting fresh.")

    saver_cb = PeriodicSaver(SAVE_DIR, name=f"xgb_fold{fold}", every=CHECKPOINT_EVERY)

    # Train (resume if checkpoint exists)
    booster = xgb.train(
        params=params,
        dtrain=dtrain,
        num_boost_round=N_BOOST_ROUNDS,
        evals=evals,
        early_stopping_rounds=EARLY_STOPPING_ROUNDS,
        callbacks=[saver_cb],
        xgb_model=resume_path if resume_path else None,
        verbose_eval=True
    )

    # Save final model (JSON, portable)
    final_json = SAVE_DIR / f"xgb_fold{fold}_final.json"
    booster.save_model(str(final_json))
    print(f"✅ Saved final model: {final_json}")

    # Predict on this fold's holdout and score
    y_prob = booster.predict(dvalid, iteration_range=(0, booster.best_iteration + 1) if booster.best_iteration is not None else None)
    y_pred = np.argmax(y_prob, axis=1)
    acc = accuracy_score(y_va, y_pred)
    fold_accs.append(acc)
    print(f"Fold {fold} accuracy: {acc:.4f}")
    if booster.best_iteration is not None:
        print(f"Best iteration (early stop): {booster.best_iteration}")

mean_acc, std_acc = np.mean(fold_accs), np.std(fold_accs)
print(f"\nMean CV accuracy: {mean_acc:.4f} ± {std_acc:.4f}")
print(f"All checkpoints & final models in: {SAVE_DIR}")


Using 5-fold CV (min per class = 7)

===== Fold 1 =====
➡️  No checkpoint found; starting fresh.
[0]	train-mlogloss:7.25759	valid-mlogloss:7.29164
[1]	train-mlogloss:7.11175	valid-mlogloss:7.17527
[2]	train-mlogloss:6.97447	valid-mlogloss:7.06605
[3]	train-mlogloss:6.84215	valid-mlogloss:6.96159
[4]	train-mlogloss:6.71501	valid-mlogloss:6.86106
[5]	train-mlogloss:6.59240	valid-mlogloss:6.76385
[6]	train-mlogloss:6.47298	valid-mlogloss:6.66891
[7]	train-mlogloss:6.35536	valid-mlogloss:6.57713
[8]	train-mlogloss:6.23946	valid-mlogloss:6.48652
[9]	train-mlogloss:6.12585	valid-mlogloss:6.39835
[10]	train-mlogloss:6.01318	valid-mlogloss:6.31035
[11]	train-mlogloss:5.90392	valid-mlogloss:6.22599
[12]	train-mlogloss:5.79519	valid-mlogloss:6.14037
[13]	train-mlogloss:5.68770	valid-mlogloss:6.05660
[14]	train-mlogloss:5.58006	valid-mlogloss:5.97246
[15]	train-mlogloss:5.47461	valid-mlogloss:5.89149
[16]	train-mlogloss:5.37075	valid-mlogloss:5.81121
[17]	train-mlogloss:5.26693	valid-mlogloss:5.7

In [ ]:
# === XGBoost CV training with resume + checkpoints + optional timing display ===
# import os, re, glob, json, pickle, time
# import numpy as np
# from pathlib import Path
# from collections import Counter
# from sklearn.model_selection import StratifiedKFold
# from sklearn.metrics import accuracy_score
# import xgboost as xgb
# from xgboost import callback

# # ---------------- CONFIG ----------------
# SAVE_DIR = Path("/content/drive/MyDrive/Stanford/CS229/Final Project/llama+kmeans_synthetic_cases_20251029-044735/xgb_checkpoints")
# SAVE_DIR.mkdir(parents=True, exist_ok=True)

# NUM_CLASS = len(le.classes_)      # from your LabelEncoder
# N_BOOST_ROUNDS = 1000             # cap on total trees
# EARLY_STOPPING_ROUNDS = 100       # stop if no improvement
# CHECKPOINT_EVERY = 250            # save every N rounds
# RANDOM_STATE = 42
# N_SPLITS = 5                      # auto-lowered if rare class < 5
# USE_TQDM = True                   # <<< OPTIONAL: set to False to disable progress bar
# # ---------------------------------------

# # ---- optional tqdm import ----
# if USE_TQDM:
#     try:
#         from tqdm.notebook import tqdm
#     except Exception:
#         !pip -q install tqdm
#         from tqdm.notebook import tqdm

# # ---- periodic saver callback ----
# class PeriodicSaver(callback.TrainingCallback):
#     """Save booster every `every` rounds to SAVE_DIR / f'{name}_{iter:04d}.json'."""
#     def __init__(self, save_dir: Path, name: str, every: int):
#         self.save_dir = Path(save_dir); self.name = name; self.every = int(every)
#     def after_iteration(self, model, epoch: int, evals_log) -> bool:
#         it = epoch + 1
#         if it % self.every == 0:
#             path = self.save_dir / f"{self.name}_{it:04d}.json"
#             model.save_model(str(path))
#             print(f"[checkpoint] saved {path}")
#         return False

# def latest_checkpoint_for_fold(save_dir: Path, fold: int):
#     """Return path to latest checkpoint (json) for this fold, or None."""
#     pattern = str(save_dir / f"xgb_fold{fold}_*.json")
#     candidates = glob.glob(pattern)
#     if not candidates: return None
#     rx = re.compile(rf"xgb_fold{fold}_(\d+)\.json$")
#     best, best_iter = None, -1
#     for p in candidates:
#         m = rx.search(os.path.basename(p))
#         if m:
#             it = int(m.group(1))
#             if it > best_iter: best, best_iter = p, it
#     return best

# def score_fold(booster: xgb.Booster, X_va, y_va):
#     dvalid = xgb.DMatrix(X_va, label=y_va)
#     best_it = getattr(booster, "best_iteration", None)
#     y_prob = booster.predict(dvalid, iteration_range=(0, best_it + 1) if best_it is not None else None)
#     y_pred = np.argmax(y_prob, axis=1)
#     return accuracy_score(y_va, y_pred), best_it

# # --- adjust #folds if rare class is too small ---
# min_per_class = min(Counter(y_train_enc).values())
# actual_splits = min(N_SPLITS, max(2, min_per_class))
# print(f"Using {actual_splits}-fold CV (min per class = {min_per_class})")

# kf = StratifiedKFold(n_splits=actual_splits, shuffle=True, random_state=RANDOM_STATE)
# fold_accs, fold_times = [], []

# # Common XGBoost params (GPU in 3.1.1 via device='cuda'; falls back to CPU if not available)
# params = {
#     "objective": "multi:softprob",
#     "num_class": NUM_CLASS,
#     "eval_metric": "mlogloss",
#     "max_depth": 4,
#     "learning_rate": 0.1,
#     "subsample": 0.9,
#     "colsample_bytree": 0.9,
#     "reg_lambda": 1.0,
#     "tree_method": "hist",   # works on both CPU and GPU in 3.x
#     "device": "cuda",        # use A100 if available
#     "seed": RANDOM_STATE,
# }

# progress = None
# if USE_TQDM:
#     progress = tqdm(total=actual_splits, desc="Cross-validation", unit="fold")

# total_start = time.time()

# for fold, (train_idx, val_idx) in enumerate(kf.split(X_train, y_train_enc), 1):
#     fold_start = time.time()
#     final_json = SAVE_DIR / f"xgb_fold{fold}_final.json"
#     print(f"\n===== Fold {fold} =====")

#     X_tr, X_va = X_train[train_idx], X_train[val_idx]
#     y_tr, y_va = y_train_enc[train_idx], y_train_enc[val_idx]

#     # If final model exists, skip training and just score (fast re-runs)
#     if final_json.exists():
#         print(f"🔁 Found completed fold model, skipping training: {final_json.name}")
#         booster = xgb.Booster(); booster.load_model(str(final_json))
#         acc, best_it = score_fold(booster, X_va, y_va)
#         fold_accs.append(acc)
#         fold_time = time.time() - fold_start
#         fold_times.append(fold_time)
#         print(f"Fold {fold} accuracy: {acc:.4f} | best_it: {best_it if best_it is not None else '-'}")
#         print(f"⏱️  Fold {fold} took {fold_time:.2f}s")
#         if progress:
#             eta = (actual_splits - fold) * (np.mean(fold_times) if fold_times else 0)
#             progress.update(1)
#             progress.set_postfix({"fold": fold, "acc": f"{acc:.4f}", "best_it": best_it if best_it is not None else "-",
#                                   "ETA (s)": f"{eta:.1f}"})
#         continue

#     dtrain, dvalid = xgb.DMatrix(X_tr, label=y_tr), xgb.DMatrix(X_va, label=y_va)
#     evals = [(dtrain, "train"), (dvalid, "valid")]

#     resume_path = latest_checkpoint_for_fold(SAVE_DIR, fold)
#     print(f"{'➡️  Resuming from ' + resume_path if resume_path else '➡️  No checkpoint found; starting fresh.'}")

#     booster = xgb.train(
#         params=params,
#         dtrain=dtrain,
#         num_boost_round=N_BOOST_ROUNDS,
#         evals=evals,
#         early_stopping_rounds=EARLY_STOPPING_ROUNDS,
#         callbacks=[PeriodicSaver(SAVE_DIR, name=f"xgb_fold{fold}", every=CHECKPOINT_EVERY)],
#         xgb_model=resume_path if resume_path else None,
#         verbose_eval=100
#     )

#     # Save final model for this fold
#     booster.save_model(str(final_json))
#     print(f"✅ Saved final model: {final_json}")

#     # Evaluate this fold
#     acc, best_it = score_fold(booster, X_va, y_va)
#     fold_accs.append(acc)
#     print(f"Fold {fold} accuracy: {acc:.4f}")
#     if best_it is not None:
#         print(f"Best iteration (early stop): {best_it}")

#     # Timing + ETA
#     fold_time = time.time() - fold_start
#     fold_times.append(fold_time)
#     eta = (actual_splits - fold) * np.mean(fold_times)
#     print(f"⏱️  Fold {fold} took {fold_time:.2f}s.  Estimated remaining ≈ {eta:.1f}s ({eta/60:.1f} min)")

#     if progress:
#         progress.update(1)
#         progress.set_postfix({"fold": fold, "acc": f"{acc:.4f}", "best_it": best_it if best_it is not None else "-",
#                               "ETA (s)": f"{eta:.1f}"})

# if progress:
#     progress.close()

# total_time = time.time() - total_start
# print(f"\nMean CV accuracy: {np.mean(fold_accs):.4f} ± {np.std(fold_accs):.4f}")
# print(f"Total training time: {total_time:.1f}s ({total_time/60:.1f} min)")
# print(f"All checkpoints & final models in: {SAVE_DIR}")


# Step 5: Evaluation with 20% validation data

In [ ]:
import xgboost as xgb
from pathlib import Path
import numpy as np

SAVE_DIR = Path("/content/drive/MyDrive/Stanford/CS229/Final Project/llama+kmeans_synthetic_cases_20251029-044735/xgb_checkpoints")

# find all final models
final_models = sorted(SAVE_DIR.glob("xgb_fold*_final.json"))
print(f"Found {len(final_models)} fold models")


Found 5 fold models


In [ ]:
dval = xgb.DMatrix(X_val, label=y_val_enc)


In [ ]:
probs = []
for path in final_models:
    booster = xgb.Booster()
    booster.load_model(str(path))
    best_it = getattr(booster, "best_iteration", None)
    y_prob = booster.predict(dval, iteration_range=(0, best_it + 1) if best_it is not None else None)
    probs.append(y_prob)

# average across folds
y_prob_mean = np.mean(probs, axis=0)
y_pred = np.argmax(y_prob_mean, axis=1)


In [ ]:
# print(y_prob_mean)
print(y_prob_mean.shape)
print(y_pred.shape)

(3320, 1660)
(3320,)


In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix

val_acc = accuracy_score(y_val_enc, y_pred)
print(f"Final validation accuracy (ensemble of 5 folds): {val_acc:.4f}")

# (Optional) confusion matrix if you want to inspect per-class results
# cm = confusion_matrix(y_val_enc, y_pred)
# print(cm)


Final validation accuracy (ensemble of 5 folds): 0.8705


# Step 6: External Evaluation from PubMed and Clinic Data


*   First process the pubmed data with Llama embedding and kmeans clustering and save it for reproducibility.
*   Run predictions using the 5-fold ensemble, output top 10/20 syndromes for each case.



In [2]:
import json
import numpy as np
import pandas as pd
from pathlib import Path
from tqdm.auto import tqdm
from scipy.sparse import csr_matrix
from sklearn.metrics import accuracy_score, top_k_accuracy_score
import xgboost as xgb

# --------------------------------------------------
# Paths
# --------------------------------------------------
BASE = "/content/drive/MyDrive/Stanford/CS229/Final Project/llama+kmeans_synthetic_cases_20251029-044735"
SAVE_DIR = Path(BASE) / "xgb_checkpoints"

PUBMED_XLSX   = Path(BASE) / "PUBMED_cohort.xlsx"
ASSIGN_CSV    = Path(BASE) / "pubmed_symptom_cluster_assignments.csv"
SYM_VOCAB_JSON = Path(BASE) / "symptom_vocab.json"
LABEL_MAP_JSON = Path(BASE) / "label_map.json"

label_col = "OMIM Diagnosis"   # ground-truth label column in PUBMED_cohort.xlsx

# --------------------------------------------------
# 1. Load data
# --------------------------------------------------
df_pub = pd.read_excel(PUBMED_XLSX)
assign_df = pd.read_csv(ASSIGN_CSV)

with open(SYM_VOCAB_JSON, "r", encoding="utf-8") as f:
    symptom_vocab_list = json.load(f)   # list of cluster_ids, len = 9883

with open(LABEL_MAP_JSON, "r", encoding="utf-8") as f:
    label_map = json.load(f)   # {disease_name: class_index}

print("PubMed rows:", len(df_pub))
print("symptom_vocab_list length:", len(symptom_vocab_list))
print("Example label_map items:", list(label_map.items())[:5])

# --------------------------------------------------
# 2. Filter PubMed rows to only labels the model knows
# --------------------------------------------------
pub_labels_raw = df_pub[label_col].astype(str)
missing_labels = [x for x in pub_labels_raw.unique() if x not in label_map]
print("Labels in PubMed not in label_map:", missing_labels)
print("Total unique missing labels:", len(missing_labels))

valid_mask = pub_labels_raw.isin(label_map.keys())
print("Total cases:", len(df_pub))
print("Valid cases:", valid_mask.sum())
print("Dropped cases:", (~valid_mask).sum())

df_pub_filtered = df_pub[valid_mask].reset_index(drop=True)
y_pubmed = df_pub_filtered[label_col].astype(str).map(label_map).values
print("df_pub_filtered shape:", df_pub_filtered.shape)
print("y_pubmed shape:", y_pubmed.shape)
print("First 10 y_pubmed:", y_pubmed[:10])

# --------------------------------------------------
# 3. Reconstruct vocab: cluster_id -> feature_index
#    symptom_vocab_list[i] = cluster_id_for_feature_i
# --------------------------------------------------
sym_vocab = {str(cluster_id): idx
             for idx, cluster_id in enumerate(symptom_vocab_list)}
print("sym_vocab size (should be 9883):", len(sym_vocab))

# --------------------------------------------------
# 4. Build symptom -> cluster_id mapping from assignment CSV
# --------------------------------------------------
def norm_text(s: str) -> str:
    s = str(s)
    s = s.strip()
    s = " ".join(s.split())   # collapse multiple spaces
    return s

assign_df["symptom_norm"] = assign_df["symptom"].apply(norm_text)
symptom_to_cluster = dict(zip(assign_df["symptom_norm"], assign_df["cluster_id"]))
print("symptom_to_cluster size:", len(symptom_to_cluster))

# quick sanity check for one known symptom
print("Example assignment row:", assign_df.head(3))

# --------------------------------------------------
# 5. Build X_pubmed_filtered (one row per PubMed case,
#    one column per feature index (0..9882), using cluster_id->feature_index mapping
# --------------------------------------------------
n_cases = len(df_pub_filtered)
vocab_size = len(sym_vocab)

indptr = [0]
indices = []
data = []

def parse_symptoms(cell):
    if pd.isna(cell):
        return []
    return [s.strip() for s in str(cell).split(";") if s.strip()]

for _, row in tqdm(df_pub_filtered.iterrows(), total=n_cases, desc="Building X_pubmed_filtered"):
    raw_syms = parse_symptoms(row["Symptoms"])

    feat_set = set()

    for s in raw_syms:
        ns = norm_text(s)
        cid = symptom_to_cluster.get(ns)   # cluster_id (0..9999)
        if cid is None:
            continue

        cid_key = str(cid)
        j = sym_vocab.get(cid_key)         # feature index (0..9882)
        if j is not None:
            feat_set.add(j)

    feat_indices = sorted(feat_set)
    indices.extend(feat_indices)
    data.extend([1] * len(feat_indices))
    indptr.append(len(indices))

X_pubmed_filtered = csr_matrix(
    (data, indices, indptr),
    shape=(n_cases, vocab_size),
    dtype=np.float32,
)

print("X_pubmed_filtered shape:", X_pubmed_filtered.shape)

# Sanity: nonzero features per row
nnz_per_row = X_pubmed_filtered.getnnz(axis=1)
print("Avg active features per case:", nnz_per_row.mean())
print("First 10 nnz counts:", nnz_per_row[:10])

# Optional: inspect first row
inv_vocab = {v: int(k) for k, v in sym_vocab.items()}  # feature_index -> cluster_id
i = 0
row0 = X_pubmed_filtered[i]
active_feats0 = row0.nonzero()[1]
active_clusters0 = [inv_vocab[j] for j in active_feats0]

print("\nRow 0 Diagnosis:", df_pub_filtered.iloc[0][label_col])
print("Row 0 Symptoms:", df_pub_filtered.iloc[0]["Symptoms"])
print("Row 0 active feature indices:", active_feats0)
print("Row 0 active cluster IDs:", active_clusters0)

# --------------------------------------------------
# 6. Load 5 XGBoost models and evaluate ensemble
# --------------------------------------------------
# Adjust the glob pattern to match your filenames, e.g. "xgb_fold*_final.json"
model_paths = sorted(SAVE_DIR.glob("xgb_fold*_final.json"))
print("Model paths found:", model_paths)

models = []
for p in model_paths:
    booster = xgb.Booster()
    booster.load_model(str(p))
    models.append(booster)
print("Loaded", len(models), "models.")

dtest = xgb.DMatrix(X_pubmed_filtered)

fold_probs = []
for model in models:
    probs = model.predict(dtest)   # shape: (n_cases, n_classes)
    fold_probs.append(probs)

fold_probs = np.stack(fold_probs, axis=0)   # (n_folds, n_cases, n_classes)
avg_probs = fold_probs.mean(axis=0)         # (n_cases, n_classes)

y_pred = np.argmax(avg_probs, axis=1)

all_classes = list(label_map.values())  # 0..1659

acc = accuracy_score(y_pubmed, y_pred)
top10 = top_k_accuracy_score(y_pubmed, avg_probs, k=10, labels=all_classes)
top20 = top_k_accuracy_score(y_pubmed, avg_probs, k=20, labels=all_classes)

print("\nEnsemble Accuracy:", acc)
print("Top-10 Accuracy:", top10)
print("Top-20 Accuracy:", top20)


PubMed rows: 101
symptom_vocab_list length: 9883
Example label_map items: [('2,4-DIENOYL-CoA REDUCTASE DEFICIENCY; DECRD', 0), ('2-METHYLBUTYRYL-CoA DEHYDROGENASE DEFICIENCY', 1), ('3-HYDROXY-3-METHYLGLUTARYL-CoA LYASE DEFICIENCY; HMGCLD', 2), ('3-HYDROXYACYL-CoA DEHYDROGENASE DEFICIENCY', 3), ('3-HYDROXYISOBUTYRYL-CoA HYDROLASE DEFICIENCY; HIBCHD', 4)]
Labels in PubMed not in label_map: ['CORONARY ARTERY DISEASE, AUTOSOMAL DOMINANT 2; ADCAD2', 'INTELLECTUAL DEVELOPMENTAL DISORDER, AUTOSOMAL DOMINANT 39; MRD39', 'INTELLECTUAL DEVELOPMENTAL DISORDER, AUTOSOMAL DOMINANT 61; MRD61', 'INFLAMMATORY BOWEL DISEASE (CROHN DISEASE) 1; IBD1', 'NEUROFIBROMATOSIS, TYPE I; NF1', 'MELANOCYTIC NEVUS SYNDROME, CONGENITAL; CMNS', 'SIFRIM-HITZ-WEISS SYNDROME; SIHIWES', 'FIBROBLAST GROWTH FACTOR 13; FGF13', 'INOSITOL POLYPHOSPHATE-4-PHOSPHATASE, TYPE I, 107-KD; INPP4A', 'CASGID SYNDROME; CASGID']
Total unique missing labels: 10
Total cases: 101
Valid cases: 90
Dropped cases: 11
df_pub_filtered shape: (90

Building X_pubmed_filtered:   0%|          | 0/90 [00:00<?, ?it/s]

X_pubmed_filtered shape: (90, 9883)
Avg active features per case: 13.78888888888889
First 10 nnz counts: [17  7 12 11 20 11  6  1  5 10]

Row 0 Diagnosis: DEVELOPMENTAL AND EPILEPTIC ENCEPHALOPATHY 94; DEE94
Row 0 Symptoms: Epilepsy (present in 92.2% of patients); Median age at seizure onset: 2 years 6 months; Multiple seizure types (generalized onset tonic-clonic, absences, myoclonic, focal-onset, atonic, myoclonic-atonic, tonic, epileptic spasms, myoclonic-clonic); Single seizure types (generalized onset tonic-clonic, absences, focal, myoclonic, myoclonic-atonic); History of febrile seizures; Status epilepticus (SE) (convulsive, nonconvulsive, or both); Prevalent epilepsy type: generalized (75.5%), combined generalized and focal (22.3%), focal (2.2%); Aggressive behavior; Attention Deficit Hyperactivity Disorder (ADHD); Autism spectrum disorder (ASD) / autistic features; Specific Learning Disorder (SLD); Obsessive-Compulsive Disorder (OCD); Psychotic symptoms; Tourette's syndrome; Bi

-------------------Do Not Use Following Codes----------------------

In [ ]:
import xgboost as xgb
from pathlib import Path
import numpy as np
from sklearn.metrics import accuracy_score, top_k_accuracy_score

SAVE_DIR = Path("/content/drive/MyDrive/Stanford/CS229/Final Project/llama+kmeans_synthetic_cases_20251029-044735/xgb_checkpoints")

# find all final models
models = sorted(SAVE_DIR.glob("xgb_fold*_final.json"))
print(f"Found {len(models)} fold models")

Found 5 fold models


In [ ]:
# Convert PubMed features into XGBoost DMatrix
import pandas as pd
import json
from scipy.sparse import csr_matrix
from tqdm.auto import tqdm

BASE_DIR = Path("/content/drive/MyDrive/Stanford/CS229/Final Project/llama+kmeans_synthetic_cases_20251029-044735")

In [ ]:
# Load PubMed cohort, cluster assignments, vocab, and label map

PUBMED_XLSX = BASE_DIR / "PUBMED_cohort.xlsx"
ASSIGN_CSV  = BASE_DIR / "pubmed_symptom_cluster_assignments.csv"
SYM_VOCAB_JSON = BASE_DIR / "symptom_vocab.json"
LABEL_MAP_JSON = BASE_DIR / "label_map.json"   # adjust if name differs

# Load data
df_pub = pd.read_excel(PUBMED_XLSX)
assign_df = pd.read_csv(ASSIGN_CSV)

with open(SYM_VOCAB_JSON, "r", encoding="utf-8") as f:
    sym_vocab = json.load(f)   # e.g. {"0": 0, "1": 1, ...}

with open(LABEL_MAP_JSON, "r", encoding="utf-8") as f:
    label_map = json.load(f)   # e.g. {"DISEASE_A": 0, "DISEASE_B": 1, ...}

print("PubMed rows:", len(df_pub))
print("Assignment rows:", len(assign_df))
print("Vocab size:", len(sym_vocab))
print("Example label_map:", list(label_map.items())[:5])
assign_df.head()


PubMed rows: 101
Assignment rows: 1205
Vocab size: 9883
Example label_map: [('2,4-DIENOYL-CoA REDUCTASE DEFICIENCY; DECRD', 0), ('2-METHYLBUTYRYL-CoA DEHYDROGENASE DEFICIENCY', 1), ('3-HYDROXY-3-METHYLGLUTARYL-CoA LYASE DEFICIENCY; HMGCLD', 2), ('3-HYDROXYACYL-CoA DEHYDROGENASE DEFICIENCY', 3), ('3-HYDROXYISOBUTYRYL-CoA HYDROLASE DEFICIENCY; HIBCHD', 4)]


,symptom,cluster_id,similarity
0,Epilepsy (present in 92.2% of patients),8039,0.753811
1,Median age at seizure onset: 2 years 6 months,4691,0.879674
2,Multiple seizure types (generalized onset toni...,9272,0.900295
3,Single seizure types (generalized onset tonic-...,615,0.870440
4,History of febrile seizures,1012,0.853744


In [ ]:
# Build a mapping: symptom → cluster_id

# Since our CSV has exactly one row per symptom, so we can do:
symptom_to_cluster = dict(zip(assign_df["symptom"], assign_df["cluster_id"]))

len(symptom_to_cluster), list(symptom_to_cluster.items())[:5]


(1205,
 [('Epilepsy (present in 92.2% of patients)', 8039),
  ('Median age at seizure onset: 2 years 6 months', 4691),
  ('Multiple seizure types (generalized onset tonic-clonic, absences, myoclonic, focal-onset, atonic, myoclonic-atonic, tonic, epileptic spasms, myoclonic-clonic)',
   9272),
  ('Single seizure types (generalized onset tonic-clonic, absences, focal, myoclonic, myoclonic-atonic)',
   615),
  ('History of febrile seizures', 1012)])

In [ ]:
import pandas as pd
from scipy.sparse import csr_matrix
from tqdm.auto import tqdm

# --- helper: text normalizer ---
def norm_text(s):
    s = str(s)
    s = s.strip()
    # collapse multiple spaces, etc.
    s = " ".join(s.split())
    return s

# --- reload / use your assignments CSV ---
assign_df = pd.read_csv(f"{BASE_DIR}/pubmed_symptom_cluster_assignments.csv")

# normalize the symptom strings from the CSV
assign_df["symptom_norm"] = assign_df["symptom"].apply(norm_text)

# build mapping: normalized symptom → cluster_id
symptom_to_cluster = dict(zip(assign_df["symptom_norm"], assign_df["cluster_id"]))
print("symptom_to_cluster size:", len(symptom_to_cluster))


symptom_to_cluster size: 1205


In [ ]:
# # choose the filtered dataframe you’re actually using
# row_idx = 0  # first row
# print("Row", row_idx, "Diagnosis:", df_pub_filtered.iloc[row_idx][label_col])
# print("Row", row_idx, "Symptoms:", df_pub_filtered.iloc[row_idx]["Symptoms"])

# def parse_symptoms(cell):
#     if pd.isna(cell):
#         return []
#     return [s.strip() for s in str(cell).split(";") if s.strip()]

# test_symptoms = parse_symptoms(df_pub_filtered.iloc[row_idx]["Symptoms"])
# print("\nParsed symptoms for row 0:")
# for s in test_symptoms:
#     ns = norm_text(s)
#     cid = symptom_to_cluster.get(ns)
#     print(f"  '{s}' -> '{ns}' -> cluster {cid}")


In [ ]:
# # Invert vocab: feature index -> cluster_id
# inv_vocab = {v: int(k) for k, v in sym_vocab.items()}

# i = 0
# row = X_pubmed_filtered[i]
# active_feats = row.nonzero()[1]
# active_clusters = [inv_vocab[j] for j in active_feats]

# print("Row", i, "active feature indices:", active_feats)
# print("Row", i, "active cluster IDs:", active_clusters)


In [ ]:
sym_vocab["2079"]

2068

In [ ]:
# Build X_pubmed (multi-hot over cluster vocab) for testing 5-fold ensemble model
n_cases = len(df_pub)
vocab_size = len(sym_vocab)

indptr = [0]
indices = []
data = []

def parse_symptoms(cell):
    if pd.isna(cell):
        return []
    return [s.strip() for s in str(cell).split(";") if s.strip()]

for _, row in tqdm(df_pub.iterrows(), total=n_cases, desc="Building X_pubmed"):
    raw_symptoms = parse_symptoms(row["Symptoms"])

    feat_set = set()

    for s in raw_symptoms:
        cid = symptom_to_cluster.get(s)
        if cid is None:
            continue

        cid_key = str(cid)
        if cid_key in sym_vocab:
            feat_set.add(sym_vocab[cid_key])   # use the feature index

    feat_indices = sorted(feat_set)
    indices.extend(feat_indices)
    data.extend([1] * len(feat_indices))
    indptr.append(len(indices))

X_pubmed = csr_matrix((data, indices, indptr), shape=(n_cases, vocab_size), dtype=float)


Building X_pubmed:   0%|          | 0/101 [00:00<?, ?it/s]

In [ ]:
# check the shape of X_pubmed

print(X_pubmed.shape)
print(np.sum(X_pubmed))

(101, 9883)
1351.0


In [ ]:
# Build y_pubmed from label_map

import json

with open(LABEL_MAP_JSON, "r", encoding="utf-8") as f:
    label_map = json.load(f)   # e.g. {"DISEASE_A": 0, "DISEASE_B": 1, ...}

print("Loaded label_map with", len(label_map), "labels.")

df_pub.columns
df_pub.head()



Loaded label_map with 1660 labels.


,Source Identifier,PMID,OMIM Diagnosis,Diagnoses,Symptoms
0,"De Maria et al., 2021 (CHD2)",34713950.0,DEVELOPMENTAL AND EPILEPTIC ENCEPHALOPATHY 94;...,CHD2-related disease; Epilepsy with myoclonic-...,Epilepsy (present in 92.2% of patients); Media...
1,"Perinelli et al., 2024 (CDKL5)",38660576.0,DEVELOPMENTAL AND EPILEPTIC ENCEPHALOPATHY 2; ...,CDKL5 deficiency disorder (CDKL5-DEE),Cortical visual impairment (CVI); Deficits in ...
2,"Whitney et al., 2023 (ST3GAL3)",37067065.0,DEVELOPMENTAL AND EPILEPTIC ENCEPHALOPATHY 15;...,ST3GAL3-related developmental and epileptic en...,Early-onset epilepsy; Seizure types: epileptic...
3,"Monfrini et al., 2023 (GABRB1)",37518907.0,DEVELOPMENTAL AND EPILEPTIC ENCEPHALOPATHY 45;...,Developmental and epileptic encephalopathy 45 ...,Hypotonia (at birth); Focal apneic seizures (a...
4,"Di Micco et al., 2024 (CACNA1E)",38780451.0,DEVELOPMENTAL AND EPILEPTIC ENCEPHALOPATHY 69;...,CACNA1E developmental and epileptic encephalop...,EEG: Polymorphic delta activity; Multifocal in...


from matplotlib import pyplot as plt
_df_0['PMID'].plot(kind='hist', bins=20, title='PMID')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
import seaborn as sns
_df_1.groupby('Source Identifier').size().plot(kind='barh', color=sns.palettes.mpl_palette('Dark2'))
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
import seaborn as sns
_df_2.groupby('OMIM Diagnosis').size().plot(kind='barh', color=sns.palettes.mpl_palette('Dark2'))
plt.gca().spines[['top', 'right',]].set_visible(False)

...

from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(_df_10['OMIM Diagnosis'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(_df_10, x='PMID', y='OMIM Diagnosis', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

<string>:5: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.



from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(_df_11['Diagnoses'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(_df_11, x='PMID', y='Diagnoses', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

<string>:5: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.



from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(_df_12['Symptoms'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(_df_12, x='PMID', y='Symptoms', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

In [ ]:
label_col = "OMIM Diagnosis"   # MUST CHANGE THIS TO MATCH DATA
pub_labels = df_pub[label_col].astype(str)

missing = [x for x in pub_labels.unique() if x not in label_map]
print("Labels in PubMed not in label_map:", missing)
print("Total Number of Labels in PubMed not in label_map:", len(missing))

# Need to drop the cases that the model never seen before.
valid_mask = pub_labels.isin(label_map.keys())
print("Total cases:", len(df_pub))
print("Valid cases:", valid_mask.sum())
print("Dropped cases:", (~valid_mask).sum())


Labels in PubMed not in label_map: ['CORONARY ARTERY DISEASE, AUTOSOMAL DOMINANT 2; ADCAD2', 'INTELLECTUAL DEVELOPMENTAL DISORDER, AUTOSOMAL DOMINANT 39; MRD39', 'INTELLECTUAL DEVELOPMENTAL DISORDER, AUTOSOMAL DOMINANT 61; MRD61', 'INFLAMMATORY BOWEL DISEASE (CROHN DISEASE) 1; IBD1', 'NEUROFIBROMATOSIS, TYPE I; NF1', 'MELANOCYTIC NEVUS SYNDROME, CONGENITAL; CMNS', 'SIFRIM-HITZ-WEISS SYNDROME; SIHIWES', 'FIBROBLAST GROWTH FACTOR 13; FGF13', 'INOSITOL POLYPHOSPHATE-4-PHOSPHATASE, TYPE I, 107-KD; INPP4A', 'CASGID SYNDROME; CASGID']
Total Number of Labels in PubMed not in label_map: 10
Total cases: 101
Valid cases: 90
Dropped cases: 11


In [ ]:
# Convert label names → integer class index using label_map
df_pub_filtered = df_pub[valid_mask].reset_index(drop=True)

y_pubmed = df_pub_filtered[label_col].astype(str).map(label_map).values
print("y_pubmed shape:", y_pubmed.shape)
print("First 10 y_pubmed:", y_pubmed[:10])



y_pubmed shape: (90,)
First 10 y_pubmed: [446 375 361 393 419 371 414 446 359 394]


In [ ]:
# verify row 0
i = 0
row = X_pubmed[i]

active_clusters = row.nonzero()[1]
print("Active cluster IDs for case 0:", active_clusters)


Active cluster IDs for case 0: [ 214  236  409  613  867 1009 1197 2079 2099 3409 3557 4642 4645 7943
 8049 9160 9842]


In [ ]:
# Filter X_pubmed to match
X_pubmed_filtered = X_pubmed[valid_mask.values]

print("X_pubmed_filtered shape:", X_pubmed_filtered.shape)
print("y_pubmed shape:", y_pubmed.shape)


X_pubmed_filtered shape: (90, 9883)
y_pubmed shape: (90,)


In [ ]:
from pathlib import Path
import xgboost as xgb

SAVE_DIR = Path("/content/drive/MyDrive/Stanford/CS229/Final Project/llama+kmeans_synthetic_cases_20251029-044735/xgb_checkpoints")

# 1) Collect the paths
model_paths = sorted(SAVE_DIR.glob("xgb_fold*_final.json"))
print("Model paths:", model_paths)

# 2) Load the boosters
models = []
for p in model_paths:
    booster = xgb.Booster()
    booster.load_model(str(p))
    models.append(booster)

print("Loaded", len(models), "models")
print("Type of first model:", type(models[0]))


Model paths: [PosixPath('/content/drive/MyDrive/Stanford/CS229/Final Project/llama+kmeans_synthetic_cases_20251029-044735/xgb_checkpoints/xgb_fold1_final.json'), PosixPath('/content/drive/MyDrive/Stanford/CS229/Final Project/llama+kmeans_synthetic_cases_20251029-044735/xgb_checkpoints/xgb_fold2_final.json'), PosixPath('/content/drive/MyDrive/Stanford/CS229/Final Project/llama+kmeans_synthetic_cases_20251029-044735/xgb_checkpoints/xgb_fold3_final.json'), PosixPath('/content/drive/MyDrive/Stanford/CS229/Final Project/llama+kmeans_synthetic_cases_20251029-044735/xgb_checkpoints/xgb_fold4_final.json'), PosixPath('/content/drive/MyDrive/Stanford/CS229/Final Project/llama+kmeans_synthetic_cases_20251029-044735/xgb_checkpoints/xgb_fold5_final.json')]
Loaded 5 models
Type of first model: <class 'xgboost.core.Booster'>


In [ ]:
dtest = xgb.DMatrix(X_pubmed)

fold_probs = [model.predict(dtest) for model in models]
fold_probs = np.stack(fold_probs)
avg_probs = fold_probs.mean(axis=0)

y_pred = np.argmax(avg_probs, axis=1)

all_classes = list(label_map.values())

acc = accuracy_score(y_pubmed, y_pred)
top10 = top_k_accuracy_score(y_pubmed, avg_probs, k=10, labels=all_classes)
top20 = top_k_accuracy_score(y_pubmed, avg_probs, k=20, labels=all_classes)

print("Accuracy:", acc)
print("Top-10:", top10)
print("Top-20:", top20)


XGBoostError: [07:24:29] /workspace/src/learner.cc:1320: Check failed: learner_model_param_.num_feature >= p_fmat->Info().num_col_ (9883 vs. 10000) : Number of columns does not match number of features in booster.
Stack trace:
  [bt] (0) /usr/local/lib/python3.12/dist-packages/xgboost/lib/libxgboost.so(+0x2bdf8c) [0x7c363dabdf8c]
  [bt] (1) /usr/local/lib/python3.12/dist-packages/xgboost/lib/libxgboost.so(+0x6dfbf9) [0x7c363dedfbf9]
  [bt] (2) /usr/local/lib/python3.12/dist-packages/xgboost/lib/libxgboost.so(+0x6f5922) [0x7c363def5922]
  [bt] (3) /usr/local/lib/python3.12/dist-packages/xgboost/lib/libxgboost.so(XGBoosterPredictFromDMatrix+0x2df) [0x7c363d9ce67f]
  [bt] (4) /lib/x86_64-linux-gnu/libffi.so.8(+0x7e2e) [0x7c36baf3fe2e]
  [bt] (5) /lib/x86_64-linux-gnu/libffi.so.8(+0x4493) [0x7c36baf3c493]
  [bt] (6) /usr/lib/python3.12/lib-dynload/_ctypes.cpython-312-x86_64-linux-gnu.so(+0x98c1) [0x7c36bc1768c1]
  [bt] (7) /usr/lib/python3.12/lib-dynload/_ctypes.cpython-312-x86_64-linux-gn

In [ ]:
# Predict with 5-fold ensemble model
from sklearn.metrics import accuracy_score, top_k_accuracy_score

dtest = xgb.DMatrix(X_pubmed_filtered)

# Predict with 5-fold ensemble model
dtest = xgb.DMatrix(X_pubmed_filtered)

fold_probs = []
for model in models:
    probs = model.predict(dtest)   # model is an xgb.Booster
    fold_probs.append(probs)

# (num_folds, n_cases, n_classes)
fold_probs = np.stack(fold_probs, axis=0)

# Average over folds (soft voting)
avg_probs = fold_probs.mean(axis=0)   # (n_cases, n_classes)

# Hard prediction (argmax over classes)
y_pred = np.argmax(avg_probs, axis=1)

# All possible class indices (0..num_classes-1) from label_map
all_classes = list(label_map.values())

# Metrics
acc = accuracy_score(y_pubmed, y_pred)

top10 = top_k_accuracy_score(
    y_pubmed,
    avg_probs,
    k=10,
    labels=all_classes,
)

top20 = top_k_accuracy_score(
    y_pubmed,
    avg_probs,
    k=20,
    labels=all_classes,
)

print("Ensemble Accuracy:", acc)
print("Top-10 Accuracy:", top10)
print("Top-20 Accuracy:", top20)

print("X_pubmed:", X_pubmed_filtered.shape)
print("y_pubmed:", y_pubmed.shape)
print("avg_probs:", avg_probs.shape)



Ensemble Accuracy: 0.0
Top-10 Accuracy: 0.03333333333333333
Top-20 Accuracy: 0.06666666666666667
X_pubmed: (90, 9883)
y_pubmed: (90,)
avg_probs: (90, 1660)


In [ ]:
print("Sample predictions:", y_pred[:40])
print("Ground truth:", y_pubmed[:40])


Sample predictions: [1010  776 1392 1532  526   80  812   80  185 1199 1032 1170   80  591
  591   80 1170   80  152  879   80  520 1392 1094  550   80  291   80
  550   80  583  359 1048   80  463  550   80   80  364 1602]
Ground truth: [ 446  375  361  393  419  371  414  446  359  394  443  427  406  373
  477  393 1510  452  375  477  452  396  357  399  366  347  405  477
  394  416  445  426  357  452 1531  477  365  440  394  443]


In [ ]:
# sym_vocab is a list where: index = feature index, value = cluster_id
sym_vocab_list = sym_vocab

# Correct dictionary: cluster_id -> feature_index
sym_vocab = { str(cluster_id): feature_index
              for feature_index, cluster_id in enumerate(sym_vocab_list) }

len(sym_vocab)

inv_vocab = {v: int(k) for k, v in sym_vocab.items()}
i = 0
print("Row", i, "Diagnosis:", df_pub_filtered.iloc[i][label_col])
print("Row", i, "Symptoms:", df_pub_filtered.iloc[i]["Symptoms"])

row = X_pubmed_filtered[i]
active_feats = row.nonzero()[1]

active_clusters = [inv_vocab[j] for j in active_feats]

print("Active feature indices:", active_feats)
print("Active cluster IDs:", active_clusters)



Row 0 Diagnosis: DEVELOPMENTAL AND EPILEPTIC ENCEPHALOPATHY 94; DEE94
Row 0 Symptoms: Epilepsy (present in 92.2% of patients); Median age at seizure onset: 2 years 6 months; Multiple seizure types (generalized onset tonic-clonic, absences, myoclonic, focal-onset, atonic, myoclonic-atonic, tonic, epileptic spasms, myoclonic-clonic); Single seizure types (generalized onset tonic-clonic, absences, focal, myoclonic, myoclonic-atonic); History of febrile seizures; Status epilepticus (SE) (convulsive, nonconvulsive, or both); Prevalent epilepsy type: generalized (75.5%), combined generalized and focal (22.3%), focal (2.2%); Aggressive behavior; Attention Deficit Hyperactivity Disorder (ADHD); Autism spectrum disorder (ASD) / autistic features; Specific Learning Disorder (SLD); Obsessive-Compulsive Disorder (OCD); Psychotic symptoms; Tourette's syndrome; Bipolar disorder; Childhood-onset schizophrenia; Brain MRI findings: normal (majority), or cerebellar hypoplasia, cerebellar vermis hypoplas





```
539|632|717|1012|1102|1890|2619|2862|9272|9298
77|109|717|1102|1697|1890|2619|2862|4997|9298
77|109|539|632|717|1012|1102|1697|1990|2619|2862|9298
77|109|539|632|717|870|1102|1990|2619|2862|4997|9298
77|109|717|870|1012|1102|1697|1890|1990|2619|2862|4997|9298
77|539|632|717|870|1102|1890|1990|2619|2862|4997|9272|9298
109|539|632|717|870|1012|1102|1697|1890|1990|2619|2862|4997|9272|9298
77|109|539|632|717|870|1102|1697|1890|1990|2619|2862|4997|9272|9298
77|109|539|632|717|870|1012|1102|1697|1890|1990|2619|2862|4997|9272|9298
[214, 236, 410, 615, 870, 1012, 1201, 2090, 2110, 3439, 3590, 4691, 4694, 8039, 8146, 9272, 9959]
```


# Step 7: Clinical Data External Evaluation

In [4]:
import json
import numpy as np
import pandas as pd
from pathlib import Path
from tqdm.auto import tqdm
from scipy.sparse import csr_matrix
from sklearn.metrics import accuracy_score, top_k_accuracy_score
import xgboost as xgb

# --------------------------------------------------
# Paths
# --------------------------------------------------
BASE = "/content/drive/MyDrive/Stanford/CS229/Final Project/llama+kmeans_synthetic_cases_20251029-044735"
SAVE_DIR = Path(BASE) / "xgb_checkpoints"

CLINICAL_XLSX    = Path(BASE) / "clinical_cohort.xlsx"
ASSIGN_CSV       = Path(BASE) / "clinical_symptom_cluster_assignments.csv"
SYM_VOCAB_JSON   = Path(BASE) / "symptom_vocab.json"
LABEL_MAP_JSON   = Path(BASE) / "label_map.json"

label_col = "OMIM-Diagnosis"   # ground-truth label column in Clinical_cohort.xlsx

# --------------------------------------------------
# 1. Load data
# --------------------------------------------------
df_clin = pd.read_excel(CLINICAL_XLSX)
assign_df = pd.read_csv(ASSIGN_CSV)

with open(SYM_VOCAB_JSON, "r", encoding="utf-8") as f:
    symptom_vocab_list = json.load(f)   # list of cluster_ids, len = 9883

with open(LABEL_MAP_JSON, "r", encoding="utf-8") as f:
    label_map = json.load(f)   # {disease_name: class_index}

print("Clinical rows:", len(df_clin))
print("symptom_vocab_list length:", len(symptom_vocab_list))
print("Example label_map items:", list(label_map.items())[:5])

# --------------------------------------------------
# 2. Filter clinical rows to only labels the model knows
# --------------------------------------------------
clin_labels_raw = df_clin[label_col].astype(str)
missing_labels = [x for x in clin_labels_raw.unique() if x not in label_map]
print("Labels in Clinical not in label_map:", missing_labels)
print("Total unique missing labels:", len(missing_labels))

valid_mask = clin_labels_raw.isin(label_map.keys())
print("Total cases (clinical):", len(df_clin))
print("Valid cases (clinical):", valid_mask.sum())
print("Dropped cases (clinical):", (~valid_mask).sum())

df_clin_filtered = df_clin[valid_mask].reset_index(drop=True)
y_clinical = df_clin_filtered[label_col].astype(str).map(label_map).values
print("df_clin_filtered shape:", df_clin_filtered.shape)
print("y_clinical shape:", y_clinical.shape)
print("First 10 y_clinical:", y_clinical[:10])

# --------------------------------------------------
# 3. Reconstruct vocab: cluster_id -> feature_index
#    symptom_vocab_list[i] = cluster_id_for_feature_i
# --------------------------------------------------
sym_vocab = {str(cluster_id): idx
             for idx, cluster_id in enumerate(symptom_vocab_list)}
print("sym_vocab size (should be 9883):", len(sym_vocab))

# --------------------------------------------------
# 4. Build symptom -> cluster_id mapping from assignment CSV
# --------------------------------------------------
def norm_text(s: str) -> str:
    s = str(s)
    s = s.strip()
    s = " ".join(s.split())   # collapse multiple spaces
    return s

assign_df["symptom_norm"] = assign_df["symptom"].apply(norm_text)
symptom_to_cluster = dict(zip(assign_df["symptom_norm"], assign_df["cluster_id"]))
print("symptom_to_cluster size (clinical):", len(symptom_to_cluster))

# quick sanity check for assignment file
print("Example assignment rows (clinical):")
print(assign_df.head(3))

# --------------------------------------------------
# 5. Build X_clinical_filtered (one row per clinical case,
#    one column per feature index (0..9882), using cluster_id->feature_index mapping
# --------------------------------------------------
n_cases = len(df_clin_filtered)
vocab_size = len(sym_vocab)

indptr = [0]
indices = []
data = []

def parse_symptoms(cell):
    if pd.isna(cell):
        return []
    return [s.strip() for s in str(cell).split(",") if s.strip()]

for _, row in tqdm(df_clin_filtered.iterrows(), total=n_cases, desc="Building X_clinical_filtered"):
    raw_syms = parse_symptoms(row["Symptoms"])   # change column name if clinical uses a different one

    feat_set = set()

    for s in raw_syms:
        ns = norm_text(s)
        cid = symptom_to_cluster.get(ns)   # cluster_id (0..9999)
        if cid is None:
            continue

        cid_key = str(cid)
        j = sym_vocab.get(cid_key)         # feature index (0..9882)
        if j is not None:
            feat_set.add(j)

    feat_indices = sorted(feat_set)
    indices.extend(feat_indices)
    data.extend([1] * len(feat_indices))
    indptr.append(len(indices))

X_clinical_filtered = csr_matrix(
    (data, indices, indptr),
    shape=(n_cases, vocab_size),
    dtype=np.float32,
)

print("X_clinical_filtered shape:", X_clinical_filtered.shape)

# Sanity: nonzero features per row
nnz_per_row = X_clinical_filtered.getnnz(axis=1)
print("Avg active features per clinical case:", nnz_per_row.mean())
print("First 10 nnz counts (clinical):", nnz_per_row[:10])

# Optional: inspect first row
inv_vocab = {v: int(k) for k, v in sym_vocab.items()}  # feature_index -> cluster_id
i = 0
row0 = X_clinical_filtered[i]
active_feats0 = row0.nonzero()[1]
active_clusters0 = [inv_vocab[j] for j in active_feats0]

print("\n[Clinical] Row 0 Diagnosis:", df_clin_filtered.iloc[0][label_col])
print("[Clinical] Row 0 Symptoms:", df_clin_filtered.iloc[0]["Symptoms"])
print("[Clinical] Row 0 active feature indices:", active_feats0)
print("[Clinical] Row 0 active cluster IDs:", active_clusters0)

# --------------------------------------------------
# 6. Load 5 XGBoost models and evaluate ensemble on clinical cohort
# --------------------------------------------------
# Adjust the glob pattern to match your filenames, e.g. "xgb_fold*_final.json"
model_paths = sorted(SAVE_DIR.glob("xgb_fold*_final.json"))
print("Model paths found:", model_paths)

models = []
for p in model_paths:
    booster = xgb.Booster()
    booster.load_model(str(p))
    models.append(booster)
print("Loaded", len(models), "models.")

dtest = xgb.DMatrix(X_clinical_filtered)

fold_probs = []
for model in models:
    probs = model.predict(dtest)   # shape: (n_cases, n_classes)
    fold_probs.append(probs)

fold_probs = np.stack(fold_probs, axis=0)   # (n_folds, n_cases, n_classes)
avg_probs = fold_probs.mean(axis=0)         # (n_cases, n_classes)

y_pred = np.argmax(avg_probs, axis=1)

all_classes = list(label_map.values())  # 0..1659

acc = accuracy_score(y_clinical, y_pred)
top10 = top_k_accuracy_score(y_clinical, avg_probs, k=10, labels=all_classes)
top20 = top_k_accuracy_score(y_clinical, avg_probs, k=20, labels=all_classes)

print("\n[Clinical] Ensemble Accuracy:", acc)
print("[Clinical] Top-10 Accuracy:", top10)
print("[Clinical] Top-20 Accuracy:", top20)


Clinical rows: 107
symptom_vocab_list length: 9883
Example label_map items: [('2,4-DIENOYL-CoA REDUCTASE DEFICIENCY; DECRD', 0), ('2-METHYLBUTYRYL-CoA DEHYDROGENASE DEFICIENCY', 1), ('3-HYDROXY-3-METHYLGLUTARYL-CoA LYASE DEFICIENCY; HMGCLD', 2), ('3-HYDROXYACYL-CoA DEHYDROGENASE DEFICIENCY', 3), ('3-HYDROXYISOBUTYRYL-CoA HYDROLASE DEFICIENCY; HIBCHD', 4)]
Labels in Clinical not in label_map: ['EPILEPSY, FAMILIAL TEMPORAL LOBE, 1; ETL1\n']
Total unique missing labels: 1
Total cases (clinical): 107
Valid cases (clinical): 102
Dropped cases (clinical): 5
df_clin_filtered shape: (102, 10)
y_clinical shape: (102,)
First 10 y_clinical: [ 184  184 1264  309  580 1655  925   76   46  754]
sym_vocab size (should be 9883): 9883
symptom_to_cluster size (clinical): 336
Example assignment rows (clinical):
                   symptom  cluster_id  similarity             symptom_norm
0           tonic seizures        1990    0.962361           tonic seizures
1  febrile seizures during        1012    0.

Building X_clinical_filtered:   0%|          | 0/102 [00:00<?, ?it/s]

X_clinical_filtered shape: (102, 9883)
Avg active features per clinical case: 6.107843137254902
First 10 nnz counts (clinical): [ 8  6  9 10  5  5  4  7  6  4]

[Clinical] Row 0 Diagnosis: CHROMOSOME 22q11.2 DELETION SYNDROME, DISTAL
[Clinical] Row 0 Symptoms: tonic seizures,  febrile seizures during, abnormal EEG, developmental delays, anxiety,Short statue, dysmorphic face,cleft palate
[Clinical] Row 0 active feature indices: [   0  302 1009 1979 2417 4945 5135 7074]
[Clinical] Row 0 active cluster IDs: [0, 303, 1012, 1990, 2435, 4997, 5188, 7156]
Model paths found: [PosixPath('/content/drive/MyDrive/Stanford/CS229/Final Project/llama+kmeans_synthetic_cases_20251029-044735/xgb_checkpoints/xgb_fold1_final.json'), PosixPath('/content/drive/MyDrive/Stanford/CS229/Final Project/llama+kmeans_synthetic_cases_20251029-044735/xgb_checkpoints/xgb_fold2_final.json'), PosixPath('/content/drive/MyDrive/Stanford/CS229/Final Project/llama+kmeans_synthetic_cases_20251029-044735/xgb_checkpoints/xgb_f

# Summary - XGBoost with 1: 9 dataset

**PubMed Data**

Ensemble Accuracy: 0.0 \
Top-10 Accuracy: 0.03333333333333333 \
Top-20 Accuracy: 0.06666666666666667

**Stanford Clinical Data**

Ensemble Accuracy: 0.0 \
Top-10 Accuracy: 0.049019607843137254 \
Top-20 Accuracy: 0.06862745098039216